In [5]:
#Load libraries
import pandas as pd
import nltk
import matplotlib
import numpy

In [33]:
df = pd.read_csv("small_df.csv")

,十六區和風料理,Shi Tzi Beef Noodles,漢神鐵板燒／新竹鐵板燒／新竹美食／新竹餐廳／新竹必吃／光復路必吃／光復路美食／光復路餐廳／園區美食／,Subway,中山道妻籠宿日式料理,Shabu Shabu (hotpot),黑白嗆 個人創意義大利美食,喜憨兒創作料理新竹建中店,開源社香雞排,明昌越式餐館,...,大中華粥麵館,新天地美食館,甘泉魚麵-清大店,曾家公園乾麵,手工現做燒餅屋 （東山總店）,如意麵食館 牛肉麵,海洋冰城/牛排,蕭家鹽酥雞,Chuan-Jia Hakka Restaurant,太河拉麵
0,4,4,5,5,4,5,4,5,3,4,...,5,5,4,4,5,5,4,4,4,4
1,"Fancy ambient, good sushi/sashimi. True that t...",The beef noodles were good and had a decent am...,Nice restaurant. A bit pricey but great food a...,I would like to mention menu for Vegetarians.\...,Pork chops a bit stiff. I ordered the pork cho...,"Very nice, friendly staff, everything is fresh...",Nice,I think that there is very tasty food here. Fu...,The food taste okay\r\nI like the sauce but th...,"The place is nice and clean, the condiment set...",...,Nice food,Really good food. Quality food and service.,tasty noodle place with variety of soup to cho...,delicious,Delicious homemade breakfast,The best beef noodle soup in Hsinchu! Highly r...,Nice place to chill and enjoy a bowl of desser...,Love it but you better call to order first,Very local restaurant near to science park ser...,The curry salty plus the chicken salty overall...
2,"[5, 'One of my favorite Japanese restaurant\nT...","[4, 'Good taste, fresh beef. Good traditional ...","[4, 'The food came in menu, with starter and d...","[3, 'Great food, but I think service can be be...","[5, ""First of all, I like the place. Will pick...","[5, 'Lots of food to select.. good for group t...","[5, ""It's delicious!\nPassed by in the past, I...","[5, ""First thing first, this restaurant contri...","[3, 'Good fried rice i enjoyed it. 😋']","[5, ""I order the food via food delivery\nThe f...",...,"[4, 'for quick time lunch.']","[5, 'Nice place, good food, enjoyed']","[5, 'I liked the noodle soup. I felt glad beca...","[4, 'Ok']","[4, 'Not bad']","[4, 'Pumpkin beef soup is delicious,noodle its...","[5, 'A bit of a weird combination (pasta and s...","[5, 'SO GOOD']","[5, 'Fine traditional Chinese restaurant. Food...","[5, 'Not just the ramen, other foods on the me..."
3,"[4, 'good japanese food\nserve in nice portion']","[5, 'Delicious and fresh taste. On a Friday ar...","[4, 'Good for casual meals. Good quality for t...","[5, 'This is the nearest Subway to Section 2 G...","[3, 'Food was not bad. Went on a weekend and t...","[4, 'Excellent for someone who can eat anythin...","[5, ""The boss is very nice and kind. He came t...","[5, 'I will be here again for the good taste f...","[4, 'Flavor is good']","[5, 'Good food, excellent customer service, no...",...,"[4, 'The price has been adjusted. The photo sh...","[5, 'Nice food']","[5, 'Very fast service. Good location as it ca...","[5, ""Right next to the park, a lot of people l...","[5, 'Traditional taste']","[5, 'Had the rolled pancake with peanut butter...","[5, 'Delicious ice especially during summer wh...","[5, 'Good']","[5, 'Authentic Hakka food. Recommend the liquo...","[3, 'A good ramen 🍜 restaurant in Hsinchu. Qu..."
4,"[3, 'Nice ambience, music’s volume can be a bi...","[4, ""i typically choose red braised 紅燒 beef no...","[5, 'Excellent food and service. There are 5 p...","[4, ""Pretty solid for Subway. You will have so...","[5, 'Good ambience... Friendly staffs & very g...","[1, ""today I have been visited this place with...","[4, 'The food is quite delicious, but when I w...","[5, 'Enjoying good food and doing the good dee...","[5, 'Perfect']","[5, 'The staffs were welcoming and friendly. T...",...,"[4, ""Side dishes are great but hard to get!\nT...","[4, 'Home made and delicious']","[4, 'Fairly priced, and love how an extra bowl...","[5, 'The sauce of the dry noodles is sweet, sa...","[3, 'Fresh']","[4, 'Nice beef noodle in reasonable price\nSpe...","[5, 'Nice Shaved Ice Cream and a large portion...","[4, 'Long queue....']","[5, 'Excellent food and an English menu. Very ...","[4, ""This place is very good. On a cold winte..."
5,"[3, 'For gathering is OK, others???']","[4, 'Integrative-sauce noodle is tasty.\nBe

Import MWE Tokenizer

In [37]:
from nltk.tokenize import MWETokenizer
mw_tokenizer = nltk.tokenize.MWETokenizer()

In [38]:
#remove headers (shop names)
df.drop(index=df.index[0], axis=0, inplace=True)

In [ ]:
#loop through csv to extract words



#split text base on space